In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import requests # library to handle requests
import csv
import lxml.html as lh
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


#!conda install --channel conda-forge geopandas --yes
import geopandas as gpd

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.datasets.samples_generator import make_blobs


# import k-means from clustering stage
from sklearn.cluster import KMeans

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

print('Libraries imported.')




Libraries imported.


In [3]:
# Fetch the file
postcode = project.get_file("crime_commute.csv")

# Read the CSV data file from the object storage into a pandas DataFrame
postcode.seek(0)
postcode=pd.read_csv(postcode)
postcode.head()

,objectid,lsoa11nm,lsoa11nmw,st_areashape,st_lengthshape,geometry,Postcode,LSOA2,Latitude,Longitude,Postcode District,crime_per_1000,public_transport_time_mins
0,5.0,Bromley 023A,Bromley 023A,569162.040237,3524.308527,POLYGON ((0.1146063324029615 51.39254569837126...,BR5 2EE,E00003519,51.389834,0.107552,BR5,126.597687,65
1,31.0,Westminster 009I,Westminster 009I,26843.034373,769.684192,POLYGON ((-0.1651940481248741 51.5223349961171...,NW1 5DF,E00023578,51.521158,-0.167708,NW1,120.926865,33
2,37.0,Newham 015B,Newham 015B,84191.765717,1413.516140,POLYGON ((0.05359712608222828 51.5390032900314...,E6 2AG,E00018285,51.540082,0.057697,E6,77.905492,40
3,38.0,Southwark 018C,Southwark 018C,242134.008988,2505.869788,POLYGON ((-0.05785501055406365 51.484316385276...,SE151RZ,E00020073,51.480043,-0.064614,SE1,126.126126,39
4,41.0,Tower Hamlets 002A,Tower Hamlets 002A,204102.320562,1785.985330,POLYGON ((-0.0525324737766623 51.5348034485191...,E2 9HJ,E00021170,51.533201,-0.053391,E2,179.050381,30


In [4]:
postcode.sort_values("Postcode District", inplace =True)
postcode.drop_duplicates(subset ="Postcode District", keep ='first', inplace = True) 
postcode.describe()

,objectid,st_areashape,st_lengthshape,Latitude,Longitude,crime_per_1000,public_transport_time_mins
count,189.000000,1.890000e+02,189.000000,189.000000,189.000000,189.000000,189.000000
mean,16959.756614,4.653640e+05,2641.715829,51.514245,-0.127938,196.820443,51.544974
std,9760.058988,1.278144e+06,1841.221067,0.081620,0.156945,501.376497,18.966497
min,67.000000,1.490576e+04,621.873334,51.299622,-0.496741,17.033357,0.000000
25%,9479.000000,1.408749e+05,1675.161621,51.472564,-0.215168,53.016453,38.000000
50%,16617.000000,2.355738e+05,2229.180651,51.518861,-0.125253,78.328982,50.000000
75%,25983.000000,3.721455e+05,2969.626089,51.576099,-0.020316,122.023810,65.000000
max,34631.000000,1.590369e+07,18553.004736,51.682138,0.209054,5032.723239,103.000000


In [5]:
# The code was removed by Watson Studio for sharing.

completed


### 5-Nearby Parks


#### Parks
>**Data source:** Foursquare API  
**Foursquare Category Id:**   
   Parks: 4bf58dd8d48988d163941735,    
   National parks: 52e81612bcbc57f1066b7a21,  
   State / Provincial Park: 5bae9231bedf3950379f89d0


In [6]:
def getNearbyParks(names, latitudes, longitudes, radius=3000):
    LIMIT=100
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4bf58dd8d48988d163941735,52e81612bcbc57f1066b7a21,5bae9231bedf3950379f89d0'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat,
            lng,
            radius, 
            LIMIT)

  
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        if len(results) > 0:
        
            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

            nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
            #print(nearby_venues)
            nearby_venues.columns = ['Town', 
                      'Town Latitude', 
                      'Town Longitude', 
                      'Venue', 
                      'Venue Latitude', 
                      'Venue Longitude', 
                      'Venue Category']
           # nearby_venues.groupby('Venue Category').count()
          #  nearby_venues.head()

    return(nearby_venues)


In [7]:
London_parks= getNearbyParks(names=postcode['Postcode District'],
                                   latitudes=postcode['Latitude'],
                                   longitudes=postcode['Longitude']
                                  )

BR1
BR2
BR3
BR4
BR5
BR6
BR7
CR0
CR2
CR3
CR4
CR5
CR6
CR7
CR8
CR9
DA1
DA5
DA6
DA7
DA8
E1 
E10
E11
E12
E13
E14
E15
E16
E17
E18
E1W
E2 
E20
E3 
E4 
E5 
E6 
E7 
E77
E8 
E9 
E98
EC1
EC2
EC4
EN1
EN2
EN3
EN4
EN5
EN8
HA0
HA1
HA2
HA3
HA4
HA5
HA6
HA7
HA8
HA9
IG1
IG2
IG3
IG4
IG5
IG6
IG7
IG8
IG9
KT1
KT2
KT3
KT4
KT5
KT6
KT9
N1 
N10
N11
N12
N13
N14
N15
N16
N17
N18
N19
N1P
N2 
N20
N21
N22
N3 
N4 
N5 
N6 
N7 
N8 
N9 
NW1
NW2
NW3
NW4
NW5
NW6
NW7
NW8
NW9
RM1
RM2
RM3
RM4
RM5
RM6
RM7
RM8
RM9
SE1
SE2
SE3
SE4
SE5
SE6
SE7
SE8
SE9
SM1
SM2
SM3
SM4
SM5
SM6
SW1
SW2
SW3
SW4
SW5
SW6
SW7
SW8
SW9
TN1
TW1
TW2
TW3
TW4
TW5
TW6
TW7
TW8
TW9
UB1
UB2
UB3
UB4
UB5
UB6
UB7
UB8
UB9
W10
W11
W12
W13
W14
W1A
W1C
W1F
W1G
W1H
W1M
W1P
W1R
W1T
W1U
W1X
W2 
W3 
W4 
W5 
W6 
W7 
W8 
W9 
WC1
WC2
WD3


In [8]:
London_parks.columns = ['Postcode District', 'Latitude','Longitude', 'Venue', 'v_lat', 'v_lng', 'cat']
#London_parks['LSOA2'] = London_parks['Postcode District'].str.title()
London_parks

,Postcode District,Latitude,Longitude,Venue,v_lat,v_lng,cat
0,BR1,51.420312,0.001354,Beckenham Place Park,51.418044,-0.012489,Park
1,BR1,51.420312,0.001354,Kelsey Park,51.401143,-0.020845,Park
2,BR1,51.420312,0.001354,Forster Memorial Park,51.432501,-0.006566,Park
3,BR1,51.420312,0.001354,Church House Gardens,51.403010,0.012149,Park
4,BR1,51.420312,0.001354,Elmstead Wood,51.422360,0.041649,Park
5,BR1,51.420312,0.001354,Beckenham Green,51.409914,-0.024944,Park
6,BR1,51.420312,0.001354,Croydon Road Recreation Ground,51.403944,-0.033028,Park
7,BR1,51.420312,0.001354,Shaftsbury Park,51.421820,0.008281,Park
8,BR1,51.420312,0.001354,Downham Fields,51.426334,0.007446,Park
9,BR1,51.420312,0.001354,Kings Meadow,51.418372,0.021050,Park


In [9]:
parks = pd.get_dummies(London_parks[['cat']], prefix="", prefix_sep="")
# add Postcode district column back to dataframe
parks['Postcode District'] = London_parks['Postcode District'] 
parks.head()


,Athletics & Sports,BBQ Joint,Basketball Court,Bridge,Building,Bus Line,Bus Stop,Café,Campground,Castle,Church,College Academic Building,Concert Hall,Event Space,Farm,Field,Forest,Garden,Golf Course,Government Building,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hotel,Industrial Estate,Lake,Law School,Monument / Landmark,Museum,Nature Preserve,Neighborhood,Observatory,Office,Other Great Outdoors,Outlet Mall,Palace,Park,Parking,Playground,Plaza,Pool,Pub,Real Estate Office,River,Road,Rugby Stadium,Scenic Lookout,Shopping Plaza,Soccer Field,Spa,Sports Club,Taxi,Tennis Court,Theme Park,Trail,Train Station,University,Veterinarian,Warehouse,Postcode District
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,BR1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,BR1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,BR1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,BR1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,BR1


In [10]:
parks_2=parks[['Postcode District', 'Campground', 'Field', 'Forest', 'Garden','Nature Preserve' , 'Park', 'Trail']]
parks_2['Total_parks'] = parks_2.sum(axis=1)
parks_2.head(100)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Postcode District,Campground,Field,Forest,Garden,Nature Preserve,Park,Trail,Total_parks
0,BR1,0,0,0,0,0,1,0,1
1,BR1,0,0,0,0,0,1,0,1
2,BR1,0,0,0,0,0,1,0,1
3,BR1,0,0,0,0,0,1,0,1
4,BR1,0,0,0,0,0,1,0,1
5,BR1,0,0,0,0,0,1,0,1
6,BR1,0,0,0,0,0,1,0,1
7,BR1,0,0,0,0,0,1,0,1
8,BR1,0,0,0,0,0,1,0,1
9,BR1,0,0,0,0,0,1,0,1


In [11]:
parks_2_grouped=parks_2.groupby('Postcode District').sum().reset_index()
parks_2_grouped

,Postcode District,Campground,Field,Forest,Garden,Nature Preserve,Park,Trail,Total_parks
0,BR1,0,0,0,0,0,21,0,21
1,BR2,0,0,0,0,0,11,0,11
2,BR3,0,0,0,0,0,20,0,20
3,BR4,0,0,0,0,0,11,0,11
4,BR5,0,0,0,0,1,4,0,5
5,BR6,0,0,0,0,0,5,0,5
6,BR7,0,0,0,0,0,7,0,7
7,CR0,0,0,0,0,0,5,0,5
8,CR2,0,0,0,0,0,8,0,8
9,CR3,0,0,0,0,0,6,0,6


In [12]:
# move Postcode Dostrict column to the first column
fixed_columns = [parks.columns[-1]] + list(parks.columns[:-1])
parks = parks[fixed_columns]

parks = pd.DataFrame(parks) 
parks_grouped=parks.groupby('Postcode District').sum().reset_index()
parks_grouped

,Postcode District,Athletics & Sports,BBQ Joint,Basketball Court,Bridge,Building,Bus Line,Bus Stop,Café,Campground,Castle,Church,College Academic Building,Concert Hall,Event Space,Farm,Field,Forest,Garden,Golf Course,Government Building,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hotel,Industrial Estate,Lake,Law School,Monument / Landmark,Museum,Nature Preserve,Neighborhood,Observatory,Office,Other Great Outdoors,Outlet Mall,Palace,Park,Parking,Playground,Plaza,Pool,Pub,Real Estate Office,River,Road,Rugby Stadium,Scenic Lookout,Shopping Plaza,Soccer Field,Spa,Sports Club,Taxi,Tennis Court,Theme Park,Trail,Train Station,University,Veterinarian,Warehouse
0,BR1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,BR2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,BR3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,20,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,BR4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,BR5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,4,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
5,BR6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,BR7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,CR0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,CR2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,CR3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
parks_grouped.describe()

,Athletics & Sports,BBQ Joint,Basketball Court,Bridge,Building,Bus Line,Bus Stop,Café,Campground,Castle,Church,College Academic Building,Concert Hall,Event Space,Farm,Field,Forest,Garden,Golf Course,Government Building,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hotel,Industrial Estate,Lake,Law School,Monument / Landmark,Museum,Nature Preserve,Neighborhood,Observatory,Office,Other Great Outdoors,Outlet Mall,Palace,Park,Parking,Playground,Plaza,Pool,Pub,Real Estate Office,River,Road,Rugby Stadium,Scenic Lookout,Shopping Plaza,Soccer Field,Spa,Sports Club,Taxi,Tennis Court,Theme Park,Trail,Train Station,University,Veterinarian,Warehouse
count,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.00000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.00000,187.000000,187.000000,187.00000,187.000000,187.00000,187.000000
mean,0.064171,0.010695,0.016043,0.042781,0.005348,0.005348,0.005348,0.016043,0.010695,0.042781,0.090909,0.005348,0.005348,0.080214,0.016043,0.016043,0.058824,1.074866,0.026738,0.010695,0.02139,0.037433,0.101604,0.133690,0.016043,0.005348,0.358289,0.016043,0.433155,0.032086,0.058824,0.893048,0.010695,0.016043,0.101604,0.005348,0.074866,28.491979,0.058824,0.358289,0.716578,0.080214,0.053476,0.005348,0.016043,0.181818,0.016043,0.171123,0.016043,0.101604,0.005348,0.005348,0.005348,0.02139,0.005348,0.181818,0.02139,0.005348,0.02139,0.005348
std,0.245715,0.103139,0.125977,0.202906,0.073127,0.073127,0.073127,0.125977,0.103139,0.202906,0.288252,0.073127,0.073127,0.272353,0.125977,0.125977,0.235926,1.692729,0.161750,0.103139,0.14507,0.190330,0.381491,0.385613,0.163165,0.073127,0.714650,0.125977,1.067392,0.176700,0.235926,1.299009,0.103139,0.219382,0.367127,0.073127,0.263882,27.261649,0.623694,0.572650,1.509786,0.272353,0.225585,0.073127,0.125977,0.386730,0.125977,0.405102,0.125977,0.302938,0.073127,0.073127,0.073127,0.14507,0.073127,0.576503,0.14507,0.073127,0.14507,0.073127
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0

In [17]:
parks_1=parks[['Postcode District','Campground', 'Field','Forest','Garden','Nature Preserve','Park', 'Trail']]
parks_1=parks_1.groupby(['Postcode District']).sum().reset_index()
parks_1.head()

,Postcode District,Campground,Field,Forest,Garden,Nature Preserve,Park,Trail
0,BR1,0,0,0,0,0,21,0
1,BR2,0,0,0,0,0,11,0
2,BR3,0,0,0,0,0,20,0
3,BR4,0,0,0,0,0,11,0
4,BR5,0,0,0,0,1,4,0


In [18]:
col_list= list(parks_grouped)
col_list.remove('Postcode District')
parks_grouped['Total_parks'] = parks_grouped[col_list].sum(axis=1)
parks_grouped

,Postcode District,Athletics & Sports,BBQ Joint,Basketball Court,Bridge,Building,Bus Line,Bus Stop,Café,Campground,Castle,Church,College Academic Building,Concert Hall,Event Space,Farm,Field,Forest,Garden,Golf Course,Government Building,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hotel,Industrial Estate,Lake,Law School,Monument / Landmark,Museum,Nature Preserve,Neighborhood,Observatory,Office,Other Great Outdoors,Outlet Mall,Palace,Park,Parking,Playground,Plaza,Pool,Pub,Real Estate Office,River,Road,Rugby Stadium,Scenic Lookout,Shopping Plaza,Soccer Field,Spa,Sports Club,Taxi,Tennis Court,Theme Park,Trail,Train Station,University,Veterinarian,Warehouse,Total_parks
0,BR1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23
1,BR2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12
2,BR3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,20,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23
3,BR4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11
4,BR5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,4,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,8
5,BR6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7
6,BR7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9
7,CR0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
8,CR2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
9,CR3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6


In [19]:
parks_total=parks_grouped[['Postcode District','Total_parks']]
parks_total.dtypes

Postcode District    object
Total_parks           int64
dtype: object

In [20]:
project.save_data(data=parks_total.to_csv(index=False),file_name='parks.csv',overwrite=True)

{'file_name': 'parks.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'dogfriendlyneigbourhoodsinlondon-donotdelete-pr-amafn7rntlwjrx',
 'asset_id': '62bc8194-5247-4299-a10f-fa7da7cb1afc'}

In [16]:
# Fetch the file
parks= project.get_file("parks.csv")

# Read the CSV data file from the object storage into a pandas DataFrame
parks.seek(0)
parks=pd.read_csv(parks)
parks.head()

,Postcode District,Total_parks
0,BR1,100
1,BR2,66
2,BR3,9
3,BR4,87
4,BR5,6


In [17]:
parks_df= pd.merge(loc_data, parks, how='right', on='Postcode District') 

parks_df.head()

,objectid,lsoa11nm,lsoa11nmw,st_areashape,st_lengthshape,geometry,Postcode,LSOA2,Latitude,Longitude,Postcode District,crime_per_1000,Total_parks
0,5,Bromley 023A,Bromley 023A,5.691620e+05,3524.308527,POLYGON ((0.1146063324029615 51.39254569837126...,BR5 2EE,E00003519,51.389834,0.107552,BR5,126.597687,6
1,1649,Bromley 029C,Bromley 029C,3.609263e+05,2755.137042,POLYGON ((0.1169602057201928 51.38070353177273...,BR5 4JP,E00003888,51.381101,0.112138,BR5,66.390041,6
2,3298,Bromley 014E,Bromley 014E,1.369637e+05,2062.999564,POLYGON ((0.1089723105937523 51.40263617316915...,BR5 3SB,E00003594,51.399822,0.109262,BR5,99.744246,6
3,3301,Bromley 014D,Bromley 014D,4.612618e+05,3509.856306,POLYGON ((0.1138637115064312 51.40059908286021...,BR5 2QH,E00003574,51.408775,0.110494,BR5,95.854922,6
4,3305,Bromley 023C,Bromley 023C,2.312132e+05,2634.221302,POLYGON ((0.1013903893214578 51.39464549031378...,BR5 2NZ,E00003597,51.396641,0.099110,BR5,35.991531,6
5,3309,Bromley 014B,Bromley 014B,4.598111e+05,3258.596330,POLYGON ((0.1103787460016984 51.39899428140831...,BR5 2LL,E00003598,51.395789,0.107495,BR5,137.049942,6
6,4013,Bromley 023B,Bromley 023B,3.679486e+05,3209.926866,POLYGON ((0.1162393713386351 51.38781684824332...,BR5 4AN,E00003529,51.384807,0.112567,BR5,81.015719,6
7,4505,Bromley 025E,Bromley 025E,4.913287e+05,3493.189679,POLYGON ((0.07732256305500072 51.3958444490719...,BR5 1LB,E00003971,51.391172,0.088094,BR5,25.357607,6
8,4977,Bromley 019E,Bromley 019E,2.649821e+06,7148.106660,POLYGON ((0.1379595985430157 51.41220421782705...,BR5 3LY,E00003501,51.398234,0.121495,BR5,162.790698,6
9,5282,Bromley 032B,Bromley 032B,1.515034e+05,1813.032251,POLYGON ((0.1134693367376506 51.37587137216043...,BR5 4QH,E00003863,51.375479,0.123144,BR5,96.482412,6
